In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
company_indgroup = pd.read_csv("company_data_with_region.csv")
company_indgroup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110606 entries, 0 to 110605
Data columns (total 30 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   Unnamed: 0                                    110606 non-null  int64  
 1   Unnamed: 0.1                                  110606 non-null  int64  
 2   Tax_number                                    110606 non-null  float64
 3   HQ_Site_Branch                                110602 non-null  object 
 4   Postal_Code                                   110501 non-null  float64
 5   City                                          110596 non-null  object 
 6   Founded                                       110387 non-null  object 
 7   Status                                        110606 non-null  object 
 8   Company_Type                                  110606 non-null  object 
 9   Liquidation_Date / Start of Legal Proceeding  38

C:\Users\Abby\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (16,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# create a list of our conditions
conditions = [
    (company_indgroup['Industry'] == 'Food Production'), 
    (company_indgroup['Industry'] == 'Manufacture of beverages'),
    (company_indgroup['Industry'] == 'Manufacture of textiles'), 
    (company_indgroup['Industry'] == 'Manufacture of wearing apparel'), 
    (company_indgroup['Industry'] == 'Manufacture of leather and related products, footwear'),
    (company_indgroup['Industry'] == 'Manufacture of wood and of products of wood and cork, except furniture;'), 
    (company_indgroup['Industry'] == 'Manufacture of paper and paper products'), 
    (company_indgroup['Industry'] == 'Printing and other reproduction activities'), 
    (company_indgroup['Industry'] == 'Furniture production'),
    (company_indgroup['Industry'] == 'Manufacture of rubber and plastic products'), 
    (company_indgroup['Industry'] == 'Manufacture of non - metallic mineral products'), 
    (company_indgroup['Industry'] == 'Manufacture of basic metals'),
    (company_indgroup['Industry'] == 'Manufacture of fabricated metal products'),
    (company_indgroup['Industry'] == 'Manufacture of computer, electronic and optical products'), 
    (company_indgroup['Industry'] == 'Manufacture of electrical equipment'),
    (company_indgroup['Industry'] == 'Manufacture of machinery and equipment nec'), 
    (company_indgroup['Industry'] == 'Manufacture of road vehicles'), 
    (company_indgroup['Industry'] == 'Manufacture of other transport equipment'),
    (company_indgroup['Industry'] == 'Other manufacturing')]

# create a list of the values we want to assign for each condition
values = ['agri', 'agri', 
          'text_leather', 'text_leather', 'text_leather', 
          'wood_paper_furni', 'wood_paper_furni', 'wood_paper_furni', 'wood_paper_furni', 
          'chemicals', 'chemicals',
          'basic_metals', 'basic_metals',
          'comp_electr', 'comp_electr',
          'machinery','machinery','machinery', 
          'other']

# create a new column and use np.select to assign values to it using our lists as arguments
company_indgroup['sector'] = np.select(conditions, values, 'non-mnfct')

# check results
company_indgroup.sector.unique()

array(['chemicals', 'non-mnfct', 'machinery', 'basic_metals', 'other',
       'text_leather', 'wood_paper_furni', 'comp_electr', 'agri'],
      dtype=object)

In [4]:
company_indgroup.to_csv('check_company.csv')

In [5]:
# IMPORTANT ASSUMPTION 1: every company's HQ is recorded in the company data set. No firms have only site branches. 
# In other words, the line below would not remove companies for the lack of HQ branch. (Only 3 tax_numbers don't have HQ)
# IMPORTANT ASSUMPTION 2: In the financial data set, every company has only 1 record of financial values each year. 
# In other words, when matching up company info and financials, it would be 1:1 relationship each year. (true)

# select columns for new company data frame
company = company_indgroup[['Tax_number', 'NUTS2', 'sector', 'HQ_Site_Branch']]
# remove duplicates
company.drop_duplicates(keep=False,inplace=True)
# remove non-manufacturers
company.drop(company[company['sector'] == 'non-mnfct'].index, inplace = True)
# remove non-HQ
company.drop(company[company['HQ_Site_Branch'] != 'HQ'].index, inplace = True)
# check results
company

C:\Users\Abby\miniconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\Abby\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Tax_number,NUTS2,sector,HQ_Site_Branch
0,1.003436e+07,Pest,chemicals,HQ
5,1.044791e+07,Southern Great Plain,basic_metals,HQ
12,1.053635e+07,Pest,basic_metals,HQ
13,1.056180e+07,Pest,machinery,HQ
15,1.056819e+07,Pest,machinery,HQ
...,...,...,...,...
110587,5.448364e+10,Western Transdanubia,basic_metals,HQ
110588,5.450419e+10,Central Transdanubia,chemicals,HQ
110590,6.201832e+10,Northern Great Plain,wood_paper_furni,HQ
110597,7.367842e+10,Southern Transdanubia,text_leather,HQ


In [6]:
# read in financial data
financial_ori = pd.read_csv("financials_index_added.csv")
financial_ori

,Unnamed: 0,Tax Number,Report Start Date,Report End Date,Net Sales (domestic revenue),Net Sales (export revenue),Total Net Sales,Revenue from other performances,Other Income,Material Cost,...,Current Liabilities,Short-Term Bank Loans,Other Short-Term Loans,Advances Received From Customers,Accounts Payable,Accrued Charges and Deferred Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses
0,0,2.902999e+10,2003-01-01,2003-12-31,NaN,NaN,7.412200e+07,0.0,0.0,NaN,...,4255000.0,NaN,NaN,NaN,NaN,29000.0,11945000.0,-0.166158,6.847178,0.940576
1,1,2.902999e+10,2004-01-01,2004-12-31,NaN,NaN,6.854100e+07,0.0,1000.0,NaN,...,5835000.0,NaN,NaN,NaN,NaN,49000.0,13144000.0,-0.012804,5.458850,0.926395
2,2,2.902999e+10,2005-01-01,2005-12-31,NaN,NaN,4.386400e+07,0.0,2000.0,NaN,...,5217000.0,NaN,NaN,NaN,NaN,29000.0,12547000.0,0.020784,3.446015,0.895747
3,3,2.902999e+10,2006-01-01,2006-12-31,NaN,NaN,4.574000e+07,0.0,1000.0,NaN,...,4488000.0,NaN,NaN,NaN,NaN,0.0,12196000.0,0.046318,3.480052,0.892872
4,4,2.902999e+10,2007-01-01,2007-12-31,NaN,NaN,3.145200e+07,0.0,18000.0,NaN,...,1855000.0,NaN,NaN,NaN,NaN,0.0,9969000.0,0.049393,2.815890,0.903597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270101,270101,1.067762e+10,2010-01-01,2010-12-31,NaN,NaN,4.430710e+08,0.0,30599000.0,NaN,...,95666000.0,NaN,NaN,NaN,NaN,401000.0,335556000.0,0.190130,2.594849,0.764046
270102,270102,1.067762e+10,2011-01-01,2011-12-31,NaN,NaN,6.295330e+08,0.0,15922000.0,NaN,...,160113000.0,NaN,NaN,NaN,NaN,868000.0,443005000.0,0.222642,2.261791,0.807806
270103,270103,1.067762e+10,2012-01-01,2012-12-31,NaN,NaN,9.127680e+08,0.0,29280000.0,NaN,...,162046000.0,NaN,NaN,NaN,NaN,0.0,506574000.0,0.254319,4.418036,0.846470
270104,270104,1.067762e+10,2013-01-01,2013-12-31,NaN,NaN,8.801600e+08,0.0,29766000.0,NaN,...,82720000.0,NaN,NaN,NaN,NaN,822000.0,499213000.0,0.369785,3.933596,0.783422


In [7]:
# make ID column name the same
financial_ori.rename(columns = {'Tax Number':'Tax_number'}, inplace = True)
# select columns for new finance data frame
financial = financial_ori[['Tax_number', 'Report Start Date', 'Report End Date',
                'Total Net Sales', 
               'Cost of Goods Sold', 
               'Gross Income', 
               'Financial Operations Income', 
               'Regular Operations Income', 
               'Extraordinary Income', 
               'Profit After Tax IS', 
               'Profit or Loss of The Year IS', 
               'Total Assets',
               "Shareholder's Equity",
               'Long-Term Liabilities',
               'Current Liabilities',
               'Total Liabilities',
               'ROA',
               'Inventory_turnover',
               'Manufacturing_Costs_to_Total_Expenses']]
# remove duplicates
financial.drop_duplicates(keep=False,inplace=True)
# check results
financial

C:\Users\Abby\miniconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Tax_number,Report Start Date,Report End Date,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,Regular Operations Income,Extraordinary Income,Profit After Tax IS,Profit or Loss of The Year IS,Total Assets,Shareholder's Equity,Long-Term Liabilities,Current Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses
0,2.902999e+10,2003-01-01,2003-12-31,7.412200e+07,71464000.0,-1741000.0,5000.0,-1736000.0,0.0,-1850000.0,-1850000.0,11134000.0,1161000.0,6500000.0,4255000.0,11945000.0,-0.166158,6.847178,0.940576
1,2.902999e+10,2004-01-01,2004-12-31,6.854100e+07,63874000.0,-169000.0,6000.0,-163000.0,0.0,-163000.0,-163000.0,12730000.0,760000.0,6500000.0,5835000.0,13144000.0,-0.012804,5.458850,0.926395
2,2.902999e+10,2005-01-01,2005-12-31,4.386400e+07,39257000.0,255000.0,1000.0,256000.0,0.0,256000.0,256000.0,12317000.0,801000.0,6500000.0,5217000.0,12547000.0,0.020784,3.446015,0.895747
3,2.902999e+10,2006-01-01,2006-12-31,4.574000e+07,40473000.0,566000.0,1000.0,567000.0,0.0,561000.0,561000.0,12112000.0,1208000.0,6500000.0,4488000.0,12196000.0,0.046318,3.480052,0.892872
4,2.902999e+10,2007-01-01,2007-12-31,3.145200e+07,28035000.0,530000.0,1000.0,531000.0,0.0,492000.0,492000.0,9961000.0,1614000.0,6500000.0,1855000.0,9969000.0,0.049393,2.815890,0.903597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270101,1.067762e+10,2010-01-01,2010-12-31,4.430710e+08,356854000.0,50794000.0,2848000.0,53642000.0,9000.0,50773000.0,50773000.0,267044000.0,239489000.0,0.0,95666000.0,335556000.0,0.190130,2.594849,0.764046
270102,1.067762e+10,2011-01-01,2011-12-31,6.295330e+08,480176000.0,97761000.0,-6653000.0,91108000.0,-260000.0,84568000.0,84568000.0,379839000.0,271593000.0,0.0,160113000.0,443005000.0,0.222642,2.261791,0.807806
270103,1.067762e+10,2012-01-01,2012-12-31,9.127680e+08,743202000.0,124522000.0,-4652000.0,119870000.0,-30000.0,112600000.0,112600000.0,442751000.0,325623000.0,0.0,162046000.0,506574000.0,0.254319,4.418036,0.846470
270104,1.067762e+10,2013-01-01,2013-12-31,8.801600e+08,651970000.0,167507000.0,1915000.0,169422000.0,-81000.0,160118000.0,160118000.0,433003000.0,390260000.0,0.0,82720000.0,499213000.0,0.369785,3.933596,0.783422


In [8]:
financial.isnull().sum().sort_values(ascending=True)

Tax_number                                   0
Report Start Date                            0
Report End Date                              0
Profit or Loss of The Year IS                0
Total Liabilities                            3
Total Assets                                 3
Gross Income                                17
Financial Operations Income                 91
Profit After Tax IS                        116
Shareholder's Equity                       195
ROA                                        226
Manufacturing_Costs_to_Total_Expenses      867
Regular Operations Income                  870
Total Net Sales                           1844
Cost of Goods Sold                        2417
Inventory_turnover                        3451
Long-Term Liabilities                     4805
Current Liabilities                       4805
Extraordinary Income                     77803
dtype: int64

In [9]:
# change ID column data type
financial['Tax_number'] = financial['Tax_number'].astype('category')
company['Tax_number'] = company['Tax_number'].astype('category')

C:\Users\Abby\AppData\Local\Temp/ipykernel_14436/3890091435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  financial['Tax_number'] = financial['Tax_number'].astype('category')
C:\Users\Abby\AppData\Local\Temp/ipykernel_14436/3890091435.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company['Tax_number'] = company['Tax_number'].astype('category')


In [10]:
# merge the two data frames
df = pd.merge(company, financial, how = 'inner', on = 'Tax_number')
# remove duplicates
df.drop_duplicates(keep=False,inplace=True)
# check results
df

,Tax_number,NUTS2,sector,HQ_Site_Branch,Report Start Date,Report End Date,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,...,Profit After Tax IS,Profit or Loss of The Year IS,Total Assets,Shareholder's Equity,Long-Term Liabilities,Current Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses
0,1.056819e+07,Pest,machinery,HQ,2003-01-01,2003-12-31,46109000.0,20296000.0,1531000.0,6000.0,...,1558000.0,1558000.0,4752000.0,6971000.0,0.0,3028000.0,10365000.0,0.327862,14.373938,0.446056
1,1.056819e+07,Pest,machinery,HQ,2004-01-01,2004-12-31,40669000.0,17226000.0,6428000.0,34000.0,...,6462000.0,6462000.0,9947000.0,7145000.0,0.0,2508000.0,9966000.0,0.649643,6.454103,0.367818
2,1.069589e+07,Southern Transdanubia,machinery,HQ,2003-01-01,2003-12-31,10530000.0,3330000.0,-949000.0,1000.0,...,-948000.0,-948000.0,13236000.0,9258000.0,0.0,3955000.0,13329000.0,-0.071623,inf,0.275184
3,1.069589e+07,Southern Transdanubia,machinery,HQ,2004-01-01,2004-12-31,13954000.0,2638000.0,2510000.0,-480000.0,...,1946000.0,1946000.0,14776000.0,10809000.0,0.0,3942000.0,14811000.0,0.131700,inf,0.214123
4,1.069589e+07,Southern Transdanubia,machinery,HQ,2005-01-01,2005-09-01,1027000.0,760000.0,-455000.0,121000.0,...,4666000.0,4666000.0,8200000.0,7177000.0,0.0,1046000.0,8223000.0,0.569024,inf,0.478891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99502,2.990746e+10,Central Transdanubia,chemicals,HQ,2016-01-01,2016-12-31,71737000.0,22725000.0,12179000.0,-468000.0,...,10994000.0,10994000.0,26348000.0,93547000.0,6400000.0,18511000.0,118481000.0,0.417261,5.241006,0.369524
99503,2.990746e+10,Central Transdanubia,chemicals,HQ,2017-01-01,2017-12-31,99789000.0,26048000.0,30427000.0,-439000.0,...,27161000.0,27161000.0,54238000.0,103089000.0,0.0,17565000.0,120654000.0,0.500774,7.859988,0.359451
99504,2.990746e+10,Central Transdanubia,chemicals,HQ,2018-01-01,2018-12-31,120394000.0,28529000.0,4665000.0,373000.0,...,4802000.0,4802000.0,68669000.0,127821000.0,0.0,10205000.0,138026000.0,0.069930,7.309506,0.239919
99505,2.990746e+10,Central Transdanubia,chemicals,HQ,2019-01-01,2019-12-31,63255000.0,24209000.0,-16380000.0,571000.0,...,-15925000.0,-15925000.0,63197000.0,130161000.0,0.0,605000.0,130766000.0,-0.251990,6.337435,0.295480


In [11]:
df.to_csv('NEW_merged_company_financial.csv')

In [12]:
# Check if Profit After Tax is a good indicator (not many nans)
df.isnull().sum().sort_values(ascending=True)

Tax_number                                   0
sector                                       0
HQ_Site_Branch                               0
Report Start Date                            0
Report End Date                              0
Profit or Loss of The Year IS                0
Total Liabilities                            1
Total Assets                                 1
Gross Income                                 7
Profit After Tax IS                         36
Financial Operations Income                 37
Shareholder's Equity                        88
ROA                                         96
Manufacturing_Costs_to_Total_Expenses      256
Regular Operations Income                  336
NUTS2                                      379
Total Net Sales                            705
Cost of Goods Sold                         871
Inventory_turnover                        1259
Long-Term Liabilities                     1818
Current Liabilities                       1818
Extraordinary

In [13]:
# Check if Tableau's calculation for averages skips nans
check = df.query('`Report Start Date`>="2020-01-01" and `Report Start Date`<="2020-12-31" and sector == "chemicals"')

In [14]:
check

,Tax_number,NUTS2,sector,HQ_Site_Branch,Report Start Date,Report End Date,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,...,Profit After Tax IS,Profit or Loss of The Year IS,Total Assets,Shareholder's Equity,Long-Term Liabilities,Current Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses
337,1.021022e+10,Western Transdanubia,chemicals,HQ,2020-01-01,2020-12-31,3.688990e+08,2.834340e+08,1.801800e+07,-1692000.0,...,1.516100e+07,1.516100e+07,1.901280e+08,1.161010e+08,0.0,90700000.0,2.181990e+08,0.079741,2.074737,0.780094
406,1.022055e+10,Budapest,chemicals,HQ,2020-01-01,2020-12-31,4.729658e+09,2.750952e+09,1.886795e+09,1090000.0,...,1.796436e+09,1.796436e+09,2.427391e+09,2.876242e+09,120000000.0,342893000.0,3.797214e+09,0.740069,3.186982,0.366889
681,1.025098e+10,Southern Great Plain,chemicals,HQ,2020-01-01,2020-12-31,2.118544e+10,1.485814e+10,2.210068e+09,-203158697.0,...,1.956030e+09,1.956030e+09,7.388244e+09,9.163303e+09,0.0,794726988.0,1.101886e+10,0.264749,9.809187,0.405763
844,1.026934e+10,Central Transdanubia,chemicals,HQ,2020-01-01,2020-12-31,1.152200e+07,6.489000e+06,1.758800e+08,10000.0,...,1.710670e+08,1.710670e+08,2.423900e+07,-7.621800e+07,0.0,26422000.0,-4.976200e+07,7.057511,inf,0.084409
863,1.026934e+10,Budapest,chemicals,HQ,2020-01-01,2020-12-31,1.152200e+07,6.489000e+06,1.758800e+08,10000.0,...,1.710670e+08,1.710670e+08,2.423900e+07,-7.621800e+07,0.0,26422000.0,-4.976200e+07,7.057511,inf,0.084409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98700,2.862989e+10,Pest,chemicals,HQ,2020-01-01,2020-12-31,2.603190e+08,1.196760e+08,5.746400e+07,2225000.0,...,5.577300e+07,5.577300e+07,3.375330e+08,3.178960e+08,2665000.0,40624000.0,4.201380e+08,0.165237,28.830643,0.498434
98864,2.889583e+10,Budapest,chemicals,HQ,2020-01-01,2020-12-31,0.000000e+00,2.200000e+04,-6.220000e+05,0.0,...,-6.220000e+05,-6.220000e+05,5.915000e+06,6.529000e+06,0.0,11000.0,6.540000e+06,-0.105156,inf,0.035200
98879,2.889583e+10,Pest,chemicals,HQ,2020-01-01,2020-12-31,0.000000e+00,2.200000e+04,-6.220000e+05,0.0,...,-6.220000e+05,-6.220000e+05,5.915000e+06,6.529000e+06,0.0,11000.0,6.540000e+06,-0.105156,inf,0.035200
99392,2.975881e+10,Western Transdanubia,chemicals,HQ,2020-01-01,2020-12-31,3.127900e+07,1.940600e+07,-2.190000e+06,2000.0,...,-2.244000e+06,-2.244000e+06,3.665100e+07,2.712100e+07,0.0,18853000.0,4.616800e+07,-0.061226,220.522727,0.572854


In [15]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
check.drop(columns=['Tax_number','NUTS2','sector','Report Start Date','Report End Date']).mean()

C:\Users\Abby\AppData\Local\Temp/ipykernel_14436/1133048460.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  check.drop(columns=['Tax_number','NUTS2','sector','Report Start Date','Report End Date']).mean()


Total Net Sales                         2097878485.620
Cost of Goods Sold                      1406181638.264
Gross Income                             463060634.205
Financial Operations Income                8772982.541
Regular Operations Income                471833616.746
Extraordinary Income                               NaN
Profit After Tax IS                      458183160.178
Profit or Loss of The Year IS            458183160.178
Total Assets                            1607053440.969
Shareholder's Equity                    1326438663.649
Long-Term Liabilities                    136874444.902
Current Liabilities                      335731817.526
Total Liabilities                       1949709714.294
ROA                                              0.412
Inventory_turnover                                 inf
Manufacturing_Costs_to_Total_Expenses            0.617
dtype: float64

In [16]:
check.isnull().sum()

Tax_number                                 0
NUTS2                                      2
sector                                     0
HQ_Site_Branch                             0
Report Start Date                          0
Report End Date                            0
Total Net Sales                            4
Cost of Goods Sold                         5
Gross Income                               0
Financial Operations Income                0
Regular Operations Income                  0
Extraordinary Income                     856
Profit After Tax IS                        0
Profit or Loss of The Year IS              0
Total Assets                               0
Shareholder's Equity                       0
Long-Term Liabilities                      5
Current Liabilities                        5
Total Liabilities                          0
ROA                                        0
Inventory_turnover                         5
Manufacturing_Costs_to_Total_Expenses      1
dtype: int